In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager
from itertools import accumulate

# 设置matplotlib绘图时的字体
my_font = font_manager.FontProperties(fname="/Library/Fonts/Songti.ttc")

# 统计句子长度及长度出现的频数
df = pd.read_csv('../Data/data_single.csv')
print(df.groupby('label')['label'].count())

df['length'] = df['evaluation'].apply(lambda x: len(x))
len_df = df.groupby('length').count()
sent_length = len_df.index.tolist()
sent_freq = len_df['evaluation'].tolist()

# 绘制句子长度及出现频数统计图
plt.bar(sent_length, sent_freq)
plt.title("句子长度及出现频数统计图", fontproperties=my_font)
plt.xlabel("句子长度", fontproperties=my_font)
plt.ylabel("句子长度出现的频数", fontproperties=my_font)
# plt.savefig("./句子长度及出现频数统计图.png")
plt.close()

# 绘制句子长度累积分布函数(CDF)
sent_pentage_list = [(count/sum(sent_freq)) for count in accumulate(sent_freq)]

# 绘制CDF
plt.plot(sent_length, sent_pentage_list)

# 寻找分位点为quantile的句子长度
quantile = 0.91
#print(list(sent_pentage_list))
for length, per in zip(sent_length, sent_pentage_list):
    if round(per, 2) == quantile:
        index = length
        break
print("\n分位点为%s的句子长度:%d." % (quantile, index))

# 绘制句子长度累积分布函数图
plt.plot(sent_length, sent_pentage_list)
plt.hlines(quantile, 0, index, colors="c", linestyles="dashed")
plt.vlines(index, 0, quantile, colors="c", linestyles="dashed")
plt.text(0, quantile, str(quantile))
plt.text(index, 0, str(index))
plt.title("句子长度累积分布函数图", fontproperties=my_font)
plt.xlabel("句子长度", fontproperties=my_font)
plt.ylabel("句子长度累积频率", fontproperties=my_font)
# plt.savefig("../Data/句子长度累积分布函数图.png")
plt.close()

label
正面    1908
负面    2375
Name: label, dtype: int64

分位点为0.91的句子长度:183.


In [14]:
import pickle
import numpy as np
import pandas as pd
from keras.utils import np_utils, plot_model
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 导入数据
# 文件的数据中，特征为evaluation, 类别为label.
def load_data(filepath, input_shape=20):
    df = pd.read_csv(filepath)

    # 标签及词汇表
    labels, vocabulary = list(df['label'].unique()), list(df['evaluation'].unique())

    # 构造字符级别的特征
    string = ''
    for word in vocabulary:
        string += word

    vocabulary = set(string)

    # 字典列表
    word_dictionary = {word: i+1 for i, word in enumerate(vocabulary)} # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下
    with open('word_dict.pk', 'wb') as f:
        pickle.dump(word_dictionary, f) # 序列化对象，将对象obj保存到文件file中去
    inverse_word_dictionary = {i+1: word for i, word in enumerate(vocabulary)}
    label_dictionary = {label: i for i, label in enumerate(labels)}
    with open('label_dict.pk', 'wb') as f:
        pickle.dump(label_dictionary, f)
    output_dictionary = {i: labels for i, labels in enumerate(labels)}

    vocab_size = len(word_dictionary.keys()) # 词汇表大小
    label_size = len(label_dictionary.keys()) # 标签类别数量

    # 序列填充，按input_shape填充，长度不足的按0补充
    x = [[word_dictionary[word] for word in sent] for sent in df['evaluation']]
    x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)
    y = [[label_dictionary[sent]] for sent in df['label']]
    y = [np_utils.to_categorical(label, num_classes=label_size) for label in y]
    y = np.array([list(_[0]) for _ in y])

    return x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary

# 创建深度学习模型， Embedding + LSTM + Softmax.
def create_LSTM(n_units, input_shape, output_dim, filepath):
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath)
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size + 1, output_dim=output_dim,
                        input_length=input_shape, mask_zero=True))
    model.add(LSTM(n_units, input_shape=(x.shape[0], x.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(label_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    plot_model(model, to_file='./model_lstm.png', show_shapes=True)
    model.summary()

    return model

# 模型训练
def model_train(input_shape, filepath, model_save_path):

    # 将数据集分为训练集和测试集，占比为9:1
    # input_shape = 100
    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary = load_data(filepath, input_shape)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.1, random_state = 42)

    # 模型输入参数，需要自己根据需要调整
    n_units = 100
    batch_size = 32
    epochs = 5
    output_dim = 20

    # 模型训练
    lstm_model = create_LSTM(n_units, input_shape, output_dim, filepath)
    lstm_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=1)

    # 模型保存
    lstm_model.save(model_save_path)

    N = test_x.shape[0]  # 测试的条数
    predict = []
    label = []
    for start, end in zip(range(0, N, 1), range(1, N+1, 1)):
        sentence = [inverse_word_dictionary[i] for i in test_x[start] if i != 0]
        y_predict = lstm_model.predict(test_x[start:end])
        label_predict = output_dictionary[np.argmax(y_predict[0])]
        label_true = output_dictionary[np.argmax(test_y[start:end])]
        print(''.join(sentence), label_true, label_predict) # 输出预测结果
        predict.append(label_predict)
        label.append(label_true)

    acc = accuracy_score(predict, label) # 预测准确率
    print('模型在测试集上的准确率为: %s.' % acc)


filepath = '../Data/data_single.csv'
input_shape = 180
model_save_path = './corpus_model.h5'
model_train(input_shape, filepath, model_save_path)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 180, 20)           43100     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               48400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 91,702
Trainable params: 91,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
3854/3854 [==============================] - 22s 6ms/step - loss: 0.4298 - acc: 0.8064
Epoch 2/5
3854/3854 [==============================] - 19s 5ms/step - loss: 0.2196 - acc: 0.9273
Epoch 3/5
3854/3854 [==============================] - 19s 5ms/step - loss: 

个43寸.物流很快三四天就到了.因为新房当时正装修就改约到20日送货.物流的服务真好.跟踪服务很及时值得称赞.19号电视送到家插上电源试试画质很清晰.音质也很棒.只是55寸的两个小时后就黑屏开不了机.打400电话20号来人检查后开单联系客服重发一个电视过来.26号打开换的电视用了两天.家里人都说机器不错.立体感很强.微鯨电视、电视界的新秀.但愿有良好的表现. 负面 负面
根本不用预约呀，1198直接买的，非传统品牌，心情坎坷不安，咬牙下单试试，收货验过屏幕没事就自己装，4个螺丝而已。头三次开机好慢啊，最长的几乎5分钟，赶紧拨打客服电话，可能是系统更新，后来就好了，待机开机飞快，音画满意。总算安心了。连接移动硬盘居然连原盘也能放，强啊！ 负面 负面
相当不错的电视，就是下雨天送货慢了点，等到晚上才送货 负面 负面
严重不满意。开机也慢。最重要的是没得诚信，大家自己看，一点信誉也没得，哪个买了哪个后悔，大家自己看！ 负面 负面
简单，容易操作，不过电脑没有自带电视直播软件，应用商店也不能直接下载，还要找个电脑下载到优盘上，然后再安装，很不方便。 负面 负面
没有问题，很不错 正面 正面
第一次收到屏裂了，京东马上换一台新机重新发过来，赞一个！！ 负面 负面
质量不错，服务太度满意。 正面 正面
第一次在京东买大件，刚开始心里不大踏实，怕出现意外情况，结果还是挺满意的，送货小哥把电视给送到家里，开箱验货，通电联网，一切正常后才离开，在此非常感谢！电视效果也挺好的，第一次用智能电视有些功能还没有搞清楚，送的电视挂件和会员还没有到位，希望商家尽快安排！谢谢！ 负面 负面
价格便宜品质好，屏幕看电视很高清 正面 正面
挺好的，是我喜欢的！ 正面 正面
家人都喜欢这个电视，索尼还是可以的。 正面 正面
还行，但是屏幕感觉显示有问题 负面 负面
内容较多，操作方便，轻薄，不知是否耐用，待观察 正面 负面
点，来的时候都十点半了。开机后，在电视上扫码注册账号，用微信、qq、微信、手机号试了很多次，一提交就显示空白页面，换网站操作也一样。打客服电话，客服只会重复一样的话，最后还是在线客服微微帮忙注册的。电视本身不错，画面跟音响都可以，连接联通ip电视、硬盘、电脑都没问题，播放很流畅。而且接口都在一侧，插拔很方便。自己装的底座，电视太大不是特别稳定，准备换成挂架。 负面 负面
老顾客了，

这个最重要的显示屏不很好，是京东方的，没仔细看，大意了！！！去年50寸的还可以。缺点： 1、图像不是很清晰； 2、音箱系统音质太差； 3、播放影片时不时有卡顿。 4、商家说的纤薄机身仅仅只是一条边框，电视机根本不纤薄！ 别的目前还行。 负面 负面
东西挺好的，还实惠，就是在挂的时候遇到点问题，物流也挺快的 正面 正面
不错 挺好的 就是比去年涨价不少 大趋势没办法 下次要买还会来的 负面 负面
很轻 感觉 挺单薄 不过价格也便宜 对得起 这个价格了 正面 正面
第二次购买微鲸电视了，第一次去年给爸妈买，还是原装LG屏幕，用了一年，蓝牙遥控，语音功能，系统都非常好用，喇叭声音音量和音质都比一般电视强，今年虽然涨价了但是通过比较和一年的体验还是下单了微鲸，试了两次还是满意的。 负面 负面
物流很快，有事没有装，昨天刚装好，看一晚上，真的很给力的一款电视，简洁方便的菜单，功能齐全还唱了首歌！没有选错。 正面 正面
电视不错，没有坏点，漏光也基本看不出来，看了下电视剧，有点拖影，网上换个接口就好了，暂时没试。京东的预约客服真的要给差评，没经过我同意擅自把送货时间改到星期五，害送货大哥白跑趟。这里要给送货大哥好评，30几度的天气，大中午把电视扛到5楼 负面 负面
这个心情真是无语了，还没开始用就又搞活动降价150元，真想退货。这叫消费者怎么树立品牌忠诚度，真是！ 负面 负面
送货速度很快，服务很好！电视送给爸妈，很喜欢！京东值得信赖！还犹豫的朋友们赶快下单吧！ 正面 正面
中肯的说整体上还是对得起这个价格的，但音效和画质都很一般，尤其是音效很是失望，不能用来听音乐，音质很杂，不知道是不是都是这样的? 负面 负面
送货快，但是再打微鲸电视安装热线时候，热线服务生直接给你说，要用微鲸自己挂架，如果因为挂架出了问题他们不管。弄得心里不舒服。还没用他家电视就在扯皮。感觉不好。 负面 负面
屏幕出厂带有一层塑料薄膜要撕开才能看清楚，屏幕清晰度相当高，看超清格式的视频就足够清晰了，这点倒是挺满意的，据我感官屏幕应该只有72色域，相对更高色域的屏幕稍显发白，不过这价也不好要求太多。系统流畅度不错，按着安卓手机的尿性买之前还以为这配置会卡得很呢，看来安卓电视和手机还是有差别的。总的来说，这机子物美价廉吧，就看能用多久了。 晒单： 收起 向左转 向右转 正面 正面
暂时没发现问题，正常使用，但

图像不清楚没有用几天就坏了 送货慢的老火 负面 负面
最垃圾的电视，无线网络经常链接不上，我家网络完全正常，其他电视或者手机电脑的wifi无线链接一直正常，售后垃圾，安装师傅软件安装一半承诺第二天来继续安装，之后就没来了，客服售后的解决是 ：也没办法联系他们当地这些零零碎碎工程师 负面 负面
试看了几天，感觉没有想像的那么好，微鲸会员的资源太少了，画面还算清晰，物流很快，总体感觉没有达到心里的预期，语音搜索有限，不知道能用几年， 负面 负面
刚收到货，看着很不错，用一段时间再来追评 正面 正面
一般吧，太厚了，架子也够厚，无线连接老是断 负面 负面
买的第一台微鲸电视 感觉很不错 系统很流畅 清晰度也可以 就是个人感觉遥控的时候会有短暂的延迟 外观各方面还是感觉挺好的 这个价位性价比还可以 负面 负面
物流快，电视不错，其中一个遥控器语音不行 负面 负面
台电视时的客服都是抢着回答问题和服务的，可是还没到两年，好像微鲸的初心变了，每次和客服交流对话都是和像机器人一样死板。做为客服无法为客户带来好的体验，这是作为客服的悲哀。但是微鲸的电视质量还是有保证的，拿了那么多台电视机，就只是坏个了摇控，画质和色彩也还行。就是现在涨价涨的厉害。再一个是二代的电视的脚比一代的不方便。最后祝微鲸越来越好，成长的同时别忘了初心。 负面 负面
在618选择了微鲸。首先，京东发货确实挺快的，但是送货有点恶心，差评。其次，送货不影响电视机的质量，拿到当晚就试了机子，投屏，华数，下载keep健身APP，会员看电影，都很流畅，开机速度确实有点慢……只能归结于智能机吧。而那些杜比4k什么的还没那种特殊要求，平时看看电视电影已经足够了……618活动也很划算。最后希望会员可以看更多电视电影咯，机子持续好下去咯。 负面 负面
基本上就是噱头，没有宣传中的那么好，基本没有任何改善，黑暗背景下四角全是白光，这应该就是传说中的漏光吧，omg。再就是背部的散热口，播放半个小时后能够明显感觉到很烫，很烫。再就是遥控器，缩水了，虽说评论后可以得到语音遥控器，可毕竟是迟来的爱啊。作为一个微鲸的老用户，冲着是对之前产品的良好体验才来再次购买的，而赤裸裸地标注是品牌高端系列，带来的体验确不敢恭维。 负面 负面
然发现了微鲸。对比了下，比小米乐视性价比高，硬件配置更高，原来买过kktv，但是对比后发现微鲸更好性价比高，虽然